<font size="5"><center> <b>Sandpyper: sandy beaches SfM-UAV analysis tools</b></center></font>

    
<center><img src="images/banner.png" width="80%"  /></center>

<font face="Calibri">
<br>
<font size="5"> <b> Multiscale sediment dynamics </b></font>

<br>
<font size="3"> <b> Nicolas Pucino; PhD Student @ Deakin University, Australia </b> <br>

<b>This notebook covers the following concepts:</b>

- The ProfileDynamics class
- Multiscale volumetric analysis and plotting
- Hotspot analysis
- Multiscale beachface Cluster Dynamics indices and plotting
</font>

___

# Introduction

In [1]:
%matplotlib notebook

import pickle
from sandpyper.hotspot import ProfileDynamics

pd.options.mode.chained_assignment = None  # default='warn'

C:\conda3\envs\sandpyper_env\lib\site-packages\pysal\explore\segregation\network\network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
C:\conda3\envs\sandpyper_env\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql
C:\conda3\envs\sandpyper_env\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
P=pickle.load(open(r"C:\my_packages\sandpyper\tests\test_data\test.p", "rb"))
P

# ProfileDynamics class

In [2]:
labels=["Undefined", "Small", "Medium", "High", "Extreme"]
appendix=["_deposition", "_erosion"]

In [7]:
D = ProfileDynamics(P, bins=5, method="JenksCaspall", labels=labels)

In [9]:
D.compute_multitemporal(loc_full={'mar': 'Marengo',
         'leo': 'St. Leonards'}, filter_class='sand')

C:\conda3\envs\sandpyper_env\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


dsm from leo = 6

ortho from leo = 6

dsm from mar = 9

ortho from mar = 9


NUMBER OF DATASETS TO PROCESS: 30


# Multiscale volumetrics

The __Mean Elevation Change (MEC)__ is calculated with the totality of the valid points (sand-only, beyond LoD, within the beachface) that occurr both in the pre and post dataset in each location.<br>

The __volumes of beachface change__, instead, are estimated, using the following procedure.

* Along each transect, the non-sand ponts are eliminated and replaced by a linearly interpolated value.
* We integrate the change along the cleaned transect to estimated volumetric change at each transect.
* We multiply the sum of all transect volume change by the transect spacing, to obtain a location-level beachface change estimation.

This approximation is necessary to go from transect change to beachface change.

In [ ]:
D.compute_volumetrics(lod=D.lod_df)

In [ ]:
D.plot_transects(location='mar', tr_id=10, dt=['dt_0','dt_2'], classified=False)

In [ ]:
D.plot_transects(location='mar', tr_id=10, dt=['dt_0','dt_2'], classified=True)

In [ ]:
D.plot_transect_mecs(location='leo',tr_id=28)

In [ ]:
D.plot_single_loc(["mar"],None)

In [ ]:
D.plot_mec_evolution(location_field="location",
                     loc_order=["leo","mar"])

##  Altimetric heatmaps and alongshore estimated volume changes
Here, we can display the alongshore estimated eroded/deposited volumes acrosse the surveyed beach.

__Note:__ The net volume change is estimated by sweeping the transect-specific value to the next transect.

In [ ]:
D.plot_along

# Hotspot analysis: Local Indicator of Spatial Association (LISA)

The function __LISA_site_level__ perform a Local Moran's I with False Discovery Rate (fdr) correction analysis for all the elevation change points in each survey in the dh_df table.

This function can use KNN-based, inverse distance weighted or binary distance-based spatial weight matrices. In this example, we model spatial relationships with a distance-based row standardised binary weight matrix with neighborhood radius of 35 m, in order to include two adjacent transect and some points from obliques without getting too far from the focal point.

we obtain a dataset containing the fdr threshold, local moran-s Is, p and z values and the quadrant in which each observation falls in a Moran's scatter plot, which represent High-High (HH, hotspot cluster), High-Low (HL, spatial outlier), Low-Low (LL, coldspot cluster) and Low-High (LH, spatial outlier) points.

We are interested in HH and LL clusters, which we generally call hotspots, and discard LH and HL points are sptial outliers.

In [10]:
D.LISA_site_level(mode="distance", distance_value=35)

Now that we have statistically significant hotspots of change, we can use those points to capture the most interesting areas of beachface change which we use to model behaviour.

However, there is a very important point to consider, which is relevant for our example.<br>
Our beachfaces are narrow, and we use only reliable valid points using multiple levels of filtering (LoD, beachface area, sand-only), which significantly reduce the total number of usable points in each timestep.
Because in our next step we will compute the Beachface Cluster Dynamics indices both at the location and transect scales, we need to create two different dataframes and (slightly different) transient-states classes:

* hotspot-filtered: we discard spatial outliers to capture location scale behaviour. Used for location scale BCDs.
* full: we disregard hotspot classification and retain all points beyond LoD. Used for transect scale BCDs.

This is necessary to assure that we have enough points in each transect to model their behaviours. Moreover, the hotspot classification has been run at the location scale. Therefore, the HH and LL hotspots are only relevant for location-scale BCD analysis.

The __labelled_hotspot_df__ dataframe holds all information about the LISA analysis for each point, including:
* Moran's scatterplot quadrant (HH, HL, LL, LH) (lisa_q)
* Local Moran's I (lisa_I)
* False discovery rate threshold (lisa_fdr)
* Simulated pseudo p-value and z-value (lisa_p_sim, lisa_z_value)

Moreover, the transition states, absed on the dh classification, are stored in the __markov_tag__ column.

# Multiscale Beachface Cluster Dynamics (BCDs) indices

In [ ]:
D.discretise(absolute=True, print_summary=True, lod=D.lod_df)

Extracting elevation from DSMs . . .


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

The __Empirical Beachface Cluster indices (e-BCD)__ uses weights to represent the importance of each transition.<br>
A point that transitioned from small erosion to extreme erosion has more impact on the sediment budget than if it was transitioning to a medium erosion class.<br>
The weights for each magnitude class can be specified manually using a dictionary. However, it is best to obtain an objective representation of the severity of change. As the magnitude classes are derived from the data (using Jenks-Caspall optimised natural breaks method), a simple yet robust solution is to use the medians of each magnitude class as weight.
<br>
<br>
The function __infer_weights__ takes care of this.

In [ ]:
D.infer_weights()

In [ ]:
D.BCD_compute_location("geometry","all",True, filterit='lod')

In [ ]:
D.plot_trans_matrices(relabel_dict)

 Observations

__The title__ informs about:
* mar/leo = Location codes (leo = St. Leonards; mar = Marengo
* n = Total number of points in the timeseries (valid and non-valid)
* t = Total number of timesteps
* trans = Total number of valid transitions considered

__leo__
>We chose St. Leonards as an example to demonstrates a few limitations, which are observable in this matrix.

St. Leonards is located within Port Phillip bay. This narrrow beach is not embayed, but __fetch-limited__, meaning that it doesn't receive the highly energetic Souther Ocean swell, rather, it sees its morphodynamics impacted by considerably lower waves which are locally generated from within the bay. Moreover, seagrass meadows and reefs further reduce wave imapcts on the subaerial beachface.<br>
Therefore, __changes are small__ and often below the limit of detections, thus, uncertain. If we consider that we filter points based on (1) only sand, (2) limit of detection, (3) spatial outliers, from an already narrow beach (short transects) this location is severly decimated in terms of behavioural modelling potential.<br>

In fact, the submatrices have many zeroes, which indicate a zero probability of these transitions, because these have not been recorded.<br>
Moreover, we note how most of the transitions tend to be __from lower magnitude classes to low magnitude classes.__
<br>
Another story for Marengo.

__mar__

Marengo is an open-ocean beach with a small southern section pretected by a small headland (tombolo-like). The section used for this example is also affected by sand nourishment, which injects considerable amount of sediment during the monitoring period.<br>
Given in Marengo we have roughly 4 times the number of valid observations (trans=30'786) as compared to St. Leonards (trans=7'465), the submatrices better capture the stochastic behaviour of this beachface.



### Plotting the e-BCDs

Let's plot the e-BCDs, which summarise the information contained in the submatrices.

 Empirical Beachface Cluster Dynamics (e-BCDs)

The function __BCDs_compute__ computes all the stochastic first-order transition matrices of sand dynamics, based on the sand-only hotspots of elevation change across beachface dataset, at the site level, which are the basis for the e-BCDs.

It returns 2 dataframes:
* __e-BCDs__
* __steady-state distribution__

Optionally, it also plots the transition matrices and save them in the specified output folder.


Notes: <br>
>These matrices discard all the __valid to non-valid transitions__. In other words, all transitions going from a valid point (non spatial outlier, classified as sand and beyond limit of detection) are labelled as "nnn" adn discarded.
Moreover, __the colour ramp__ higher limit (vmax parameter) is set to a maximum of 3 times the standard deviation of all dataset without the nnn state.

In [ ]:
D.plot_location_ebcds()

## Residual Beachface Cluster Dynamics (r-BCD)

We already computed the __Steady State__ probability vectors for each location using the __BCDs_compute__ function, storing it in the __ss__ variable.<br>
Here below we reorder the data and plot as an heatmap, where each columns is a location and each row a magnitude of change.


In [ ]:
ss=D.location_ss
order=[i for i in D.tags_order if i !='nnn']
ss=ss.loc[order] # use set to extract rows with common names
ss

In [ ]:
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.family'] = 'sans-serif'
sb.set_context("paper", font_scale=2)


f,ax=plt.subplots(figsize=(12,10))

sb.heatmap(ss, cmap="Blues",annot=True,
           annot_kws={'size':14},linewidths=1,linecolor="white", cbar_kws={'label': 'Lim. Probabilities'});

#f.savefig(r'E:\\path\\to\\save\\picture.png', dpi=600); 

### Computing the r-BCDs

The residual is simply the difference between erosional and depositional probabilities in the Steady-State distribution, __multiplicated by 100__ for readability purposes. Note that in this case, no weigths are applied as no transition is represented.


Here below, the dataframe returns the residual column, which is what you might want to map in Qgis.

In [ ]:
D.location_ss

The above table, the row __r_bcds__ represent the r-BCD index for the locations, which we also call __behavioural regime__.

Location level BCDs are good to compare multiple locations across wide areas. But what about getting a more detailed spatial-explicit view of the behaviour of a beachface system, at the transect-level?

two parameters are important:
* __min_points__: the minimum required valid points per transect to consider a transect reliable.
* __thresh__: the minimum number of timesteps required to retain a transect.


In [ ]:
loc_specs={'mar':{'thresh':6,
       'min_points':6}}

D.BCD_compute_transects(loc_specs=loc_specs,reliable_action='keep', dirNameTrans=D.ProfileSet.dirNameTrans)

In [ ]:
D.transects_rbcd.query("location=='leo'").plot(column="residual", cmap='RdBu_r')

# Sensitivity analysis

For a reliable computation of r-BCDs at the transect scale the Δh points within the subaerial beachface must be beyond the period-specific LoDs.

This filtering process assures that only high quality data is used in the r-BCDs computation to the detriment of the total number of valid observations per transect. Moreover, two additional filtering steps are applied at the site-level, to ensure that are only retained:
* points that remain valid for at least a certain amount of Δh periods (t)
* transects that have a number of valid points greater than a determined minimum threshold (pt)

This is done to ensure comparability across time and transects in a determined location.
Therefore, __t is an important parameter that should ideally be as high as the available time periods__, in order to ensure that the maximum behavioural variability is captured.<br>
Yet, setting this parameter very high can reduce considerably the number of valid points retained in a single transect, which in turn can fall below t, leading to the loss of transects from the final behavioural map.<br>
It is also informative to monitor and try to minimise the number of transects that __passed from a depositional to erosional behavioral regime (or vice versa) in the last time period (i. e. changed sign from t-1 to t),__ for any chosen t. Those transects could signal a behaviour that only emerged by choosing a determined value for t, signaling a potentially __lower confidence in their r-BCD values__.<br>
For these reasons, the sensitivity analysis helps in choosing a __sub-optimal combination of t and p such as__:
* at least 85% (arbitrary) of the total transects are retained
* a reasonably low number of sign changes occurred

We start by defining all the combinations between the parameters __thresh (t)__ and __min_pts (pt)__, given ranges that we define.<br>
The thresh parameter can be up to the total number of timesteps available, while the min_pts, we decide to test the values from 0 to 210, with a step of 10.

In [ ]:
ss_tr_big = sensitivity_tr_rbcd(D.df_labelled,
                                test_thresholds='max',
                                test_min_pts=[0,20,2])

In [ ]:
f,ax=plt.subplots(figsize=(10,10))

palette=sb.color_palette( n_colors=ss_tr_big.tr_id.unique().shape[0])
sb.lineplot(data=ss_tr_big, x='thresh',y='residual', hue='tr_id', style='location',
            palette=palette, legend=False, **dict(alpha=0.5),
            ax=ax
)
ax.set_ylabel("r_bcd")
ax.axhline(y=0, lw=2, c='r');

As we can see, there are only 3 transects that flipped their r-BCD sign (depo to ero or viceversa) as a consequence of an increase in the minimum number of timesteps.

In order to better decide a sub-optimal combination of thresh and min_pts that retains the majority of the transects (while keeping an eye on transects that flipped r-BCD), here below we do some plotting.

First, some preprocessing...

In [ ]:
plot_sensitivity_rbcds_transects(ss_tr_big, location='mar')

# Conclusion

This lenghty notebook showed what Sandpyper needs to operate and what the input data should look like. In the next notebooks I will show how all this information will be analysed in a few lines of code using Sandpyper.